<a href="https://colab.research.google.com/github/DouglasDiasEng/Action-Recognition-/blob/main/Guia_passo_a_passo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalação das Bibliotecas

Para começar, instalamos duas bibliotecas essenciais: **OpenCV** e **TensorFlow Hub**, que serão usadas para processar os vídeos e carregar o modelo pré-treinado de reconhecimento de ações.



In [2]:
!pip install -q opencv-python tensorflow_hub

## Importação das Bibliotecas

Neste trecho, importamos todas as bibliotecas necessárias para processamento de vídeo, carregamento do modelo, manipulação de arquivos temporários e exibição de resultados no notebook.
O print exibi a versão atual do TensorFlow instalada no ambiente, garantindo que é compatível com o modelo a ser carregado.

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
from google.colab.patches import cv2_imshow
import re
import os
import tempfile
import ssl
import cv2
import numpy as np
from urllib import request
from IPython.display import HTML
from base64 import b64encode

print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.19.0


## Carregando os Rótulos do Kinetics-400

O modelo I3D utiliza a lista de ações do dataset **Kinetics-400**, que contém 400 classes diferentes.  
Neste trecho, fazemos o download dessa lista diretamente do repositório oficial e a carregamos em uma lista Python.


In [4]:
url_classes = 'https://raw.githubusercontent.com/deepmind/kinetics-i3d/master/data/label_map.txt'
with request.urlopen(url_classes) as file:
  label = [row.decode('utf-8').strip() for row in file.readlines()]

print("Total de classes: ", len(label))
print("Algumas classes: ", label[:10])

Total de classes:  400
Algumas classes:  ['abseiling', 'air drumming', 'answering questions', 'applauding', 'applying cream', 'archery', 'arm wrestling', 'arranging flowers', 'assembling computer', 'auctioning']
